## Ebay Web Scraper

useful links for info info on parsing html and the libraries/functions used here:  
https://www.w3schools.com/xml/xpath_syntax.asp   - xPath syntax - find nodes in xml/html  
https://lxml.de/lxmlhtml.html - 

questions, comments, ect:  
    -how are headers used, how would I change them
    -What is the HTML get request object?
        -what is the object created by html.fromstring from this?
    -where does xpath come from?
    -why does """if __name__=="__main__":""" return true


In [139]:
import argparse #allows for command line arguments insertion
from pprint import pprint #not used???
from traceback import format_exc 

import requests
import unicodecsv as csv
from lxml import html
import pandas as pd
import numpy as np

from PIL import Image #not stricktly necesarry for scraping
from io import BytesIO # see above comment


**parse** function takes the *brand* (search query) argument, creates a html get request with ebay url formula, then useds xpath to sort through the page and first find listings - then price, product url, and title from within the listings. Each listing is written to a dictionary with it's info, and the dictionary then appended to the list of listings.

In [184]:
def parse(brand):

    url = 'https://www.ebay.com/sch/i.html?_nkw={0}&_sacat=0&LH_PrefLoc=1'.format(brand)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
    failed = False

    # Retries for handling network errors
    for _ in range(5):
        #print ("Retrieving %s"%(url))
        response = requests.get(url, headers=headers, verify=False) #use requests to send a html get request - scrape the ebay page
        parser = html.fromstring(response.text) 
        #print ("Parsing page")

        if response.status_code!=200:
            failed = True
            continue
        else:
            failed = False
            break

    if failed:
        return []

    product_listings = parser.xpath('//li[contains(@id,"results-listing")]') #use xpath to find all listing nodes
    raw_result_count = parser.xpath("//h1[contains(@class,'count-heading')]//text()")
    result_count = ''.join(raw_result_count).strip()
    #print ("Found {0} for {1}".format(result_count,brand))
    scraped_products = []

    for product in product_listings:
        raw_url = product.xpath('.//a[contains(@class,"item__link")]/@href')
        raw_title = product.xpath('.//h3[contains(@class,"item__title")]//text()')
        raw_product_type = product.xpath('.//h3[contains(@class,"item__title")]/span[@class="LIGHT_HIGHLIGHT"]/text()')
        raw_price = product.xpath('.//span[contains(@class,"s-item__price")]//text()')
        price  = ' '.join(' '.join(raw_price).split())
        title = ' '.join(' '.join(raw_title).split())
        product_type = ''.join(raw_product_type)
        title = title.replace(product_type, '').strip()
        data = {
                    'url':raw_url[0],
                    'title':title,
                    'price':price
        }
        scraped_products.append(data)
    return scraped_products


## below cell does not work inside a jupyter notebook
the argparser stuff allows for command line arguments (ie. >python scrape bn94*** from cl)  
Takes data list and writes it to a csv file named '''arg-ebay-scraped-data.csv'''

In [4]:
if __name__=="__main__": #this returns true, but idk why

    argparser = argparse.ArgumentParser()
    argparser.add_argument('brand',help = 'Brand Name')
    args = argparser.parse_args()
    brand = args.brand

    scraped_data =  parse(brand)
    if scraped_data: # if data scrape successful
        print ("Writing scraped data to %s-ebay-scraped-data.csv"%(brand)) #output log
        with open('%s-ebay-scraped-data.csv'%(brand),'wb') as csvfile: #open csv file to be written to in binary
            fieldnames = ["title","price","url"] #collumns
            writer = csv.DictWriter(csvfile,fieldnames = fieldnames,quoting=csv.QUOTE_ALL)
            writer.writeheader()
            for data in scraped_data:
                writer.writerow(data)
    else:
        print("No data scraped")

usage: ipykernel_launcher.py [-h] brand
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

C:\Users\isaac\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [186]:
search = list(input().split(','))
arr = np.empty((len(list(search)),5))

for idx,i in enumerate(search):
    dat = parse(i)
    table = pd.DataFrame(data=dat)
    #print(table)
    prices = table['price'].str.strip('$ ')
    
    for pidx,pi in enumerate(prices):
        try:
            prices.iloc[pidx] = float(pi)
        except:
            prices.iloc[pidx] = None      #prices.iloc[idx].split('$')[1]
            #prices.iloc[idx] = prices.iloc[idx].split('$')[1] pick higher of a price range
            
    #rel = prices.mean(),prices.std(),prices.max(),prices.min()
    #mean = prices.mean()
    #std = prices.std()
    #max_ = prices.max()
    #min_ = prices.min()
    
    arr[idx,:] = prices.mean(), prices.mode()[0], prices.std(), prices.max(), prices.min() #  search,mean,std,max_,min_

cols = ['mean','mode','standard dev','max','min']

pd.DataFrame(arr,columns=cols,index=search)
  



 bn44-00787a,bn44-00707a


C:\Users\isaac\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\isaac\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,mean,mode,standard dev,max,min
bn44-00787a,47.010000,11.99,26.871442,99.00,7.00
bn44-00707a,46.374478,8.28,40.622047,239.99,4.46


In [ ]:
## make an array instead
if __name__=="__main__": #this returns true, but idk why

    argparser = argparse.ArgumentParser()
    argparser.add_argument('brand',help = 'Brand Name')
    args = argparser.parse_args()
    brand = args.brand

    scraped_data =  parse(brand)
    if scraped_data: # if data scrape successful
        print ("Writing scraped data to %s-ebay-scraped-data.csv"%(brand)) #output log
        
        with open('%s-ebay-scraped-data.csv'%(brand),'wb') as csvfile: #open csv file to be written to in binary
            fieldnames = ["title","price","url"] #collumns
            writer = csv.DictWriter(csvfile,fieldnames = fieldnames,quoting=csv.QUOTE_ALL)
            writer.writeheader()
            for data in scraped_data:
                writer.writerow(data)
    else:
        print("No data scraped")

In [119]:
#req.status_code
#req.encoding
#req.text #get text from request - using encoding specified by req.encoding
#req.content 
#i = Image.open(BytesIO(req.content)
lnk = 'https://www.ebay.com/sch/i.html?_nkw={0}&_sacat=0'.format('samsung')

sponse = requests.get(lnk, verify=False) #use requests to send a html get request - scrape the ebay page



#parser = html.fromstring(response.text) 


#html.fromstring(sponse.text).xpath('//li[contains(@id,"results-listing")]')#.xpath('.//a[contains(@class,"item__link")]/@href')[0]
sponse.content

C:\Users\isaac\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


b'<!DOCTYPE html><!--[if IE 9]><html class="ie9" lang="en"><![endif]--><!--[if gt IE 9]><!--><html lang="en"><!--<![endif]--><head><meta http-equiv="X-UA-Compatible" content="IE=edge"><script>"use strict";if(window.PerformanceObserver&&performance&&performance.mark&&performance.getEntriesByName){window.SRP=window.SRP||{};var paintObserver=new window.PerformanceObserver(function(e){var r=e.getEntries();r.sort(function(e,r){return e.startTime-r.startTime});var n=r[1].startTime;window.SRP.TTI_TIMER={lastInteractiveWindow:n};var t=new window.PerformanceObserver(function(e){for(var r=e.getEntries(),i=0,a=r.length;i<a;i++)r[i].startTime-n>=5e3&&(window.SRP.TTI_TIMER.timeToInteract=n,t.disconnect()),n=r[i].startTime+r[i].duration,window.SRP.TTI_TIMER.lastInteractiveWindow=n});t.observe({entryTypes:["longtask"]}),paintObserver.disconnect()});paintObserver.observe({entryTypes:["paint"]})};</script><meta name="msvalidate.01" content="34E98E6F27109BE1A9DCF19658EEEE33" /><meta name="yandex-verific